In [72]:
from keras import Model, Input
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split


In [58]:
DATASET_PATH = "../../data/raw/training.csv"
IMGS_SIZE = (96, 96)

In [59]:
data = pd.read_csv(DATASET_PATH)
print("before drop na", len(data.index))
data = data.fillna(method="ffill")
print("after drop na", len(data.index))

before drop na 7049
after drop na 7049


In [60]:
img_array = np.array([data['Image'][i].split(' ') for i in range(len(data))],dtype='float')
img_array = np.array([img.reshape(96,96,1) for img in img_array])

In [61]:
keypoints = data.drop(["Image"], axis=1).sort_index()

In [67]:
X_train, X_test, Y_train, Y_test = train_test_split(img_array, keypoints, test_size=0.2 ,shuffle=True)

In [77]:
len(X_test)

1410

In [62]:
EPOCHS = 100
BATCH_SIZE = 64

In [63]:
input = Input(shape=(96, 96, 1))
x = Conv2D(32, (3, 3), activation="relu")(input)
x = MaxPooling2D()(x)
x = Conv2D(16, (3, 3), activation="relu")(x)
x = MaxPooling2D()(x)
x = Conv2D(8, (3, 3), activation="relu")(x)
x = Flatten()(x)
output = Dense(30, activation="relu")(x)
model = Model(inputs=input, outputs=output)
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 96, 96, 1)]       0         
                                                                 
 conv2d_9 (Conv2D)           (None, 94, 94, 32)        320       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 47, 47, 32)       0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 45, 45, 16)        4624      
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 22, 22, 16)       0         
 2D)                                                             
                                                                 
 conv2d_11 (Conv2D)          (None, 20, 20, 8)         1160

In [68]:
model.compile(optimizer="adam", loss="mean_absolute_error")
vat_acc_checkpoint = ModelCheckpoint(
    "best_model.h5", monitor="val_loss", mode="min", save_best_only=True
)

In [70]:
hostory = model.fit(img_array,
                    keypoints,
                    validation_split=0.2, 
                    epochs=EPOCHS, 
                    batch_size=BATCH_SIZE,
                    callbacks=[vat_acc_checkpoint])

Epoch 1/100
89/89 [==============================] - 0s 5ms/step - loss: 0.9143 - val_loss: 1.1762
Epoch 2/100
89/89 [==============================] - 0s 5ms/step - loss: 0.9072 - val_loss: 1.1920
Epoch 3/100
89/89 [==============================] - 0s 5ms/step - loss: 0.9127 - val_loss: 1.1436
Epoch 4/100
89/89 [==============================] - 0s 5ms/step - loss: 0.9019 - val_loss: 1.1738
Epoch 5/100
89/89 [==============================] - 0s 5ms/step - loss: 0.9071 - val_loss: 1.1626
Epoch 6/100
89/89 [==============================] - 0s 4ms/step - loss: 0.8977 - val_loss: 1.1557
Epoch 7/100
89/89 [==============================] - 0s 5ms/step - loss: 0.9216 - val_loss: 1.1226
Epoch 8/100
89/89 [==============================] - 0s 4ms/step - loss: 0.9009 - val_loss: 1.1371
Epoch 9/100
89/89 [==============================] - 0s 5ms/step - loss: 0.8919 - val_loss: 1.1834
Epoch 10/100
89/89 [==============================] - 0s 4ms/step - loss: 0.8968 - val_loss: 1.2089
Epoch 11/

In [79]:
model = load_model("best_model.h5")
model.evaluate(X_test, Y_test)

45/45 [==============================] - 0s 2ms/step - loss: 0.8874


0.8874320983886719

In [75]:
pred = model.predict([X_test[0]])

ValueError: in user code:

    File "/home/vkrotov/my_ml_projects/venv/lib/python3.8/site-packages/keras/engine/training.py", line 1845, in predict_function  *
        return step_function(self, iterator)
    File "/home/vkrotov/my_ml_projects/venv/lib/python3.8/site-packages/keras/engine/training.py", line 1834, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/vkrotov/my_ml_projects/venv/lib/python3.8/site-packages/keras/engine/training.py", line 1823, in run_step  **
        outputs = model.predict_step(data)
    File "/home/vkrotov/my_ml_projects/venv/lib/python3.8/site-packages/keras/engine/training.py", line 1791, in predict_step
        return self(x, training=False)
    File "/home/vkrotov/my_ml_projects/venv/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None

    ValueError: Exception encountered when calling layer "conv2d_9" (typ